Updated: 05/02/24

The purpose of this workbook is to provide several additional tests (beyond mitman & cheung workbooks) of quadratic modes in the ringdown. 


### Tests in this workbook

1. Do the fits independently in (l,m) modes and check for amplitude ratio consistency.  
2. Do the fits at different CCE levels and look for consistency. 


Tests (other)

1. Plot the mismatch at different times during the ringdown ($t_0$) with and without the quadratic mode. This can be done for a single (l,m) or the full data. 
2. Plot the amplitude of the quadratic mode at different times during the ringdown ($t_0$). If it is really there it should decay in line with the decay time. 
3. Plot the quadratic amplitude [for (2,2,0,1)x(2,2,0,1)] against the linear amplitude (2,2,0,1) for a number of different simulations and look for quadratic scaling compared to the amplitude of the (4,4,0,1) mode, which shouldn't have the same relationship (see Mitman et al.).
5. Let the frequency vary for the quadratic mode in the model and plot the mismatch.
7. Try fits with (2,2,0,1)x(2,2,0,1) and (2,-2,0,-1)x(2,-2,0,-1). These should be mirror images, however noise may not be mirrored (though, it may be). 

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import qnmfitsrd as qnmfits
from multiprocessing import Pool
from spatial_reconstruction_tests.spatial_reconstruction import *
from matplotlib.animation import FuncAnimation
from scipy.interpolate import UnivariateSpline
from Visualisation.qnm_vis import * 
from qnmfitsrd.CCE_file_getter import *
import datetime
from scipy.optimize import least_squares

l_max = 5
n_max = 5

In [4]:
CCE_sim1 = CCE_to_sim_simplified(sim_num='0001')
CCE_sim2 = CCE_to_sim_simplified(sim_num='0002')
CCE_sim3 = CCE_to_sim_simplified(sim_num='0003')
CCE_sim4 = CCE_to_sim_simplified(sim_num='0004')
CCE_sim5 = CCE_to_sim_simplified(sim_num='0005')
CCE_sim6 = CCE_to_sim_simplified(sim_num='0006')
CCE_sim7 = CCE_to_sim_simplified(sim_num='0007')
CCE_sim8 = CCE_to_sim_simplified(sim_num='0008')
CCE_sim9 = CCE_to_sim_simplified(sim_num='0009')
CCE_sim10 = CCE_to_sim_simplified(sim_num='0010')
CCE_sim11 = CCE_to_sim_simplified(sim_num='0011')
CCE_sim12 = CCE_to_sim_simplified(sim_num='0012')
CCE_sim13 = CCE_to_sim_simplified(sim_num='0013')

CCE_sim305 = CCE_to_sim_simplified(sim_num='0305')

In [43]:
print(CCE_sim305.metadata)

{'simulation_name': 'BBH_SKS_d14.3_q1.22_sA_0_0_0.330_sB_0_0_-0.440/Lev6', 'alternative_names': ['PRIVATE:BBH:0225', 'SXS:BBH:0305'], 'initial_data_type': 'BBH_SKS', 'object_types': 'BHBH', 'number_of_orbits': 15.168887885, 'reference_mass_ratio': 1.2212026034520165, 'reference_chi_eff': -0.0166486688481975, 'reference_chi1_perp': 3.352575606023737e-08, 'reference_chi2_perp': 6.555433639961451e-08, 'reference_eccentricity': 0.0008382, 'reference_dimensionless_spin1': [3.41456411037e-08, -4.09141073094e-08, 0.329976580046], 'reference_dimensionless_spin2': [3.75379590194e-08, 3.17672731622e-08, -0.43994832522], 'reference_orbital_frequency': [1.05119123869e-09, -5.9266081495e-10, 0.0179961458572], 'reference_mass1': 0.549802229834, 'reference_mass2': 0.450213771474, 'reference_time': 540.0, 'reference_position1': [-6.46530157405, -0.386662502327, -8.5250406227e-06], 'reference_position2': [7.94532387193, 0.388267452553, -9.34127387124e-06], 'reference_mean_anomaly': 3.765934, 'keywords'

In [5]:
sim_1502 = qnmfits.SXS(ID=1502, zero_time=(2,2))
sim_1476 = qnmfits.SXS(ID=1476, zero_time=(2,2))
sim_1506 = qnmfits.SXS(ID=1506, zero_time=(2,2))
sim_1508 = qnmfits.SXS(ID=1508, zero_time=(2,2))
sim_1474 = qnmfits.SXS(ID=1474, zero_time=(2,2))
sim_1505 = qnmfits.SXS(ID=1505, zero_time=(2,2))
sim_1504 = qnmfits.SXS(ID=1504, zero_time=(2,2))
sim_1485 = qnmfits.SXS(ID=1485, zero_time=(2,2))
sim_1486 = qnmfits.SXS(ID=1486, zero_time=(2,2))
sim_1441 = qnmfits.SXS(ID=1441, zero_time=(2,2))
sim_1500 = qnmfits.SXS(ID=1500, zero_time=(2,2))
sim_1492 = qnmfits.SXS(ID=1492, zero_time=(2,2))
sim_1465 = qnmfits.SXS(ID=1465, zero_time=(2,2))
sim_1458 = qnmfits.SXS(ID=1458, zero_time=(2,2))
sim_1438 = qnmfits.SXS(ID=1438, zero_time=(2,2))
sim_1430 = qnmfits.SXS(ID=1430, zero_time=(2,2))

/data/rvnd2/python_envs/python-3.11-clone/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Skipping download from 'https://data.black-holes.org/catalog.json' because local file is newer
Found the following files to load from the SXS catalog:
    SXS:BBH:1502v5/Lev3/metadata.json
Found the following files to load from the SXS catalog:
    SXS:BBH:1502v5/Lev3/rhOverM_Asymptotic_GeometricUnits_CoM.h5


100%|██████████| 83.0M/83.0M [00:14<00:00, 6.13MB/s]


Found the following files to load from the SXS catalog:
    SXS:BBH:1476v5/Lev3/metadata.json
Found the following files to load from the SXS catalog:
    SXS:BBH:1476v5/Lev3/rhOverM_Asymptotic_GeometricUnits_CoM.h5


100%|██████████| 80.4M/80.4M [00:13<00:00, 6.13MB/s]


...

100%|██████████| 80.8M/80.8M [00:12<00:00, 6.54MB/s]


Found the following files to load from the SXS catalog:
    SXS:BBH:1438v5/Lev3/metadata.json
Found the following files to load from the SXS catalog:
    SXS:BBH:1438v5/Lev3/rhOverM_Asymptotic_GeometricUnits_CoM.h5


100%|██████████| 85.1M/85.1M [00:13<00:00, 6.46MB/s]


Found the following files to load from the SXS catalog:
    SXS:BBH:1430v5/Lev3/metadata.json
Found the following files to load from the SXS catalog:
    SXS:BBH:1430v5/Lev3/rhOverM_Asymptotic_GeometricUnits_CoM.h5


100%|██████████| 88.3M/88.3M [00:14<00:00, 6.36MB/s]


# Amplitude ratios

Linear case

In [42]:
sim = CCE_sim305

t0 = 0

model_l1 = [(2,2,n,1) for n in range(2+1)] + [(3,2,n,1) for n in range(2+1)] + [(4,2,n,1) for n in range(2+1)] 

index_220 = model_l1.index((2,2,0,1))

best_fit_22 = qnmfits.ringdown_fit(
    sim.times,
    sim.h[2,2],
    model_l1,
    Mf=sim.Mf,
    chif=sim.chif_mag,
    t0=t0,
)

best_fit_32 = qnmfits.ringdown_fit(
    sim.times,
    sim.h[3,2],
    model_l1,
    Mf=sim.Mf,
    chif=sim.chif_mag,
    t0=t0,
)

best_fit_42 = qnmfits.ringdown_fit(
    sim.times,
    sim.h[4,2],
    model_l1,
    Mf=sim.Mf,
    chif=sim.chif_mag,
    t0=t0,
)

c_220_22 = abs(best_fit_22['C'][index_220])
c_220_32 = abs(best_fit_32['C'][index_220])
c_220_42 = abs(best_fit_42['C'][index_220])

spherical_modes = [(2,2), (3,2), (4,2)]

indices_lists_l = [[lm_mode+mode for mode in [(2,2,0,1)]] for lm_mode in spherical_modes]
mu_lists = [qnm.mu_list(indices, sim.chif_mag) for indices in indices_lists_l]

print("Ratio of 22 to 32")
print("Amplitudes: ", c_220_22/c_220_32)
print("Mixing coefficients: ", abs(mu_lists[0][0]/mu_lists[1][0]))

print("Ratio of 22 to 42")
print("Amplitudes: ", c_220_22/c_220_42)
print("Mixing coefficients: ", abs(mu_lists[0][0]/mu_lists[2][0]))




Ratio of 22 to 32
Amplitudes:  14.206749598142537
Mixing coefficients:  14.292870121829791
Ratio of 22 to 42
Amplitudes:  263.39609685900086
Mixing coefficients:  264.01094252780905


In [41]:
sim = CCE_sim305

t0 = 20

model_l1 = [(4,4,n,1) for n in range(2+1)] + [(5,4,n,1) for n in range(2+1)] + [(6,4,n,1) for n in range(2+1)] + [(2,2,0,1,2,2,0,1)]

index_220220 = model_l1.index((2,2,0,1,2,2,0,1))

best_fit_22 = qnmfits.ringdown_fit(
    sim.times,
    sim.h[4,4],
    model_l1,
    Mf=sim.Mf,
    chif=sim.chif_mag,
    t0=t0,
)

best_fit_32 = qnmfits.ringdown_fit(
    sim.times,
    sim.h[5,4],
    model_l1,
    Mf=sim.Mf,
    chif=sim.chif_mag,
    t0=t0,
)

best_fit_42 = qnmfits.ringdown_fit(
    sim.times,
    sim.h[6,4],
    model_l1,
    Mf=sim.Mf,
    chif=sim.chif_mag,
    t0=t0,
)

c_220220_44 = abs(best_fit_22['C'][index_220220])
c_220220_54 = abs(best_fit_32['C'][index_220220])
c_220220_64 = abs(best_fit_42['C'][index_220220])

spherical_modes = [(4,4), (5,4), (6,4)]

indices_lists_q = [[lm_mode+mode for mode in [(2,2,0,1,2,2,0,1)]] for lm_mode in spherical_modes]
alpha_lists = [qnm.alpha(indices, sim.chif_mag) for indices in indices_lists_q]

print("Ratio of 44 to 54")
print("Amplitudes: ", c_220220_44/c_220220_54)
print("Mixing coefficients: ", abs(alpha_lists[0][0]/alpha_lists[1][0]))

print("Ratio of 44 to 64")
print("Amplitudes: ", c_220220_44/c_220220_64)
print("Mixing coefficients: ", abs(alpha_lists[0][0]/alpha_lists[2][0]))


Ratio of 44 to 54
Amplitudes:  8.373504394248371
Mixing coefficients:  1.2100321881914708
Ratio of 44 to 64
Amplitudes:  17.349129627752795
Mixing coefficients:  1.659391443751505
